<p style="font-size:20px; color:orange; text-align:center"><b>This is a project to get started with recommendation systems!</b></p>

<p style="font-size:18px; color:orange;">Below we can see the types of recommender systems, In this notebook the focus is going to be on Content-based systems</p>

<img src='https://www.alpha-quantum.com/blog/wp-content/uploads/2020/05/img_5ed4354677fb7.png'>



# *Import and Explore*

In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import warnings
warnings.filterwarnings("ignore")

/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv


In [2]:
movies_org=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')

In [3]:
movies=movies_org.copy()

In [4]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


We are going to focus on the overview column from movies dataset, and create a recommendation system based on this


In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

# *Duplicates in the Original Title*

*There are two duplicates in the original title, if they remain unchanged, they're going to cause an error later (in recommend_movies function), so I'm going to add the releasing year to the original title.*

In [6]:
movies.original_title.loc[movies.original_title.duplicated()]

3693    Out of the Blue
4267             Batman
Name: original_title, dtype: object

In [7]:
movies.original_title.iloc[3693]='Out of the Blue 2006'

In [8]:
movies.original_title.iloc[4267]='Batman 1966'

# *Vectorization and Sigmoid Kernel*

*We need to create a vector for each overview (each movie) so we can compute similarity between movies, but before that, let's fill nan values in the overview column, with an empty string.*

In [9]:
movies.overview=movies.overview.fillna(" ")

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv=TfidfVectorizer(min_df=3, max_features=None,strip_accents='unicode',token_pattern=r'\w{1,}',ngram_range=(1,3),stop_words='english')

In [11]:
tfv_matrix=tfv.fit_transform(movies.overview)

In [12]:
tfv_matrix.shape

(4803, 10417)

*Now let's find correlation between the vectors for each pair of overviews*

## *A Little About Sigmoid Kernel*

*The function sigmoid_kernel computes the sigmoid kernel between two vectors. The sigmoid kernel is also known as hyperbolic tangent, or Multilayer Perceptron (because, in the neural network field, it is often used as neuron activation function). It is defined as:*


*where:*

<img src='https://nixustechnologies.com/wp-content/uploads/2022/03/the-sigmoid-kernel-function.webp'>

*x, y are the input vectors*

a *is known as slope*

c *is known as intercept*

In [13]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig=sigmoid_kernel(tfv_matrix,tfv_matrix)

In [14]:
sig[1]

array([0.76159416, 0.76163447, 0.76159416, ..., 0.76159513, 0.76159416,
       0.76159416])

*Next we're writing a function to take a movie name as input and returns similar movies based on the overview.*

*Here's what this function does:*

1. *Finds the index of the movie name, using original_title column*

1. *Finds the values obtained from sigmoid kernel for that index (remember, sigmoid kernel returns a n * n matrix, n= number of movies)*

1. *We now have the similarity scores between the input movie and all the other movies (including itself) so it now enumerates the scores, so we know which score belongs to which index.*

1. *It sorts the scores, remember that we have the the scores and the indices*

1. *It makes a list of indices of the most similar movies*

1. *It returns the movies belonging to those indices*

In [15]:
def recommend_movies(movie):
    idx=movies.loc[movies.original_title==movie,:].index[0]
    recom=list(enumerate(sig[idx]))
    top_recom=sorted(recom, key=lambda x:x[1],reverse=True)[1:6]
    top_idx=[i[0] for i in top_recom]
    top_movies=movies.original_title.iloc[top_idx]
    top_movies.reset_index(drop=True,inplace=True)
    return top_movies

In [16]:
recommend_movies('Batman')

0    The Dark Knight Rises
1            Batman Begins
2           Batman & Robin
3           Batman Forever
4           Batman Returns
Name: original_title, dtype: object

# *Reference*
[This YT video](https://www.youtube.com/watch?v=i4a0Of22QRg&list=PLZoTAELRMXVN7QGpcuN-Vg35Hgjp3htvi&index=6)